# Topic Modeling Assessment Project

Welcome to your Topic Modeling Assessment! For this project you will be working with a dataset of over 400,000 quora questions that have no labeled cateogry, and attempting to find optimal number of cateogries to assign these questions to. The .csv file of these text questions can be found in the NLP folder.


#### Task: Import pandas and read in the quora_questions.csv file.

In [55]:
import pandas as pd
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

//anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
//anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
quora = pd.read_csv('quora_questions.csv')

In [3]:
quora.head()

,Question
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."


In [4]:
quora.shape

(404289, 1)

# Preprocessing

#### Task: Create a vectorized document term matrix. 

- How do you want to clean up your text with regards to stopwords, special characters, and other situations.
- Using a Countvectorizer versus a TFIDFvectorizer
- You may want to explore the max_df and min_df parameters. 


In [28]:
quoras = quora.sample(n=40429)

In [7]:
import re 
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizer(text):
    # remove html tags from all of the text before processing
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    # Creating our token object, which is used to create documents with linguistic annotations.
    # we disabled the parser and ner parts of the pipeline in order to speed up parsing
    mytokens = nlp(cleantext, disable=['parser', 'ner'])

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.90, min_df=10, stop_words='english')

In [29]:
%%time
dtm = cv.fit_transform(quoras['Question'])

CPU times: user 1min 11s, sys: 486 ms, total: 1min 11s
Wall time: 1min 12s


In [30]:
dtm

<40429x2948 sparse matrix of type '<class 'numpy.int64'>'
	with 158397 stored elements in Compressed Sparse Row format>

# LDA Modelling

#### TASK: Using Scikit-Learn create an instance of LDA. 

- You can manually run and tune your model, then evaluate the resulting clusters. 
- Or you can use gridsearch to try and identify the best number of topics to use. 


In [13]:
from sklearn.decomposition import LatentDirichletAllocation

In [44]:
lda_model = LatentDirichletAllocation(n_components=15,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=8192,            # n docs in each learning iter
                                      learning_decay=.7,
                                      evaluate_every= -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=8192, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=20,
                          mean_change_tol=0.001, n_components=15, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [46]:
%%time
lda_output = lda_model.fit_transform(dtm)

CPU times: user 21.3 s, sys: 1.33 s, total: 22.6 s
Wall time: 30.7 s


In [22]:
#First run with sample of 20000
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -517266.3915213088
Perplexity:  1246.533130127213
{'batch_size': 1024, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 20, 'mean_change_tol': 0.001, 'n_components': 20, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [33]:
#second run with sample of ~40k or 10% of the set
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -1209147.1267954083
Perplexity:  1626.996158395568
{'batch_size': 4096, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 20, 'mean_change_tol': 0.001, 'n_components': 20, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [47]:
#third run with gridsearch params
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -1197747.6249968878
Perplexity:  1517.4363076037073
{'batch_size': 8192, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 20, 'mean_change_tol': 0.001, 'n_components': 15, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


#### Task: Evaluate the different models you have run and determine which model you think determines the best clusters.  


The evaluation part could invlove:
- Printing out the top 15 most common words for each of the topics and seeing if they make sense.
- Using the perplexity and log-likelihoood scores.
- Using the pyLDAvis tool to investigate the different clusters. 

In [48]:
len(cv.get_feature_names())

2948

In [49]:
len(lda_model.components_)

15

In [19]:
# first run with sample of 20000
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['review', 'today', 'process', 'music', 'tip', 'history', 'code', 'idea', 'major', 'university', 'look', 'compare', 'country', 'world', 'new']


THE TOP 15 WORDS FOR TOPIC #1
['support', 'fast', 'right', 'read', 'song', 'free', 'google', 'watch', 'ask', 'friend', 'answer', 'book', 'good', 'question', 'quora']


THE TOP 15 WORDS FOR TOPIC #2
['join', 'law', 'rate', 'mechanical', 'united', 'download', 'career', '2017', 'water', 'iphone', '2', 'big', 'place', 'good', 'india']


THE TOP 15 WORDS FOR TOPIC #3
['bangalore', 'available', 'offer', 'market', 'reduce', 'american', 'term', 'test', 'easy', 'earn', 'service', 'online', 'money', 'good', 'way']


THE TOP 15 WORDS FOR TOPIC #4
['network', 'apple', 'technology', 'date', 'charge', 'window', 'marketing', 'web', 'chinese', 'cost', 'social', 'run', '10', 'play', 'buy']


THE TOP 15 WORDS FOR TOPIC #5
['event', 'eye', 'turn', 'able', 'california', 'development', 'value', 'chance', 'end', 'design', 'human', 'car

In [34]:
len(lda_model.components_)

20

In [35]:
#second run with 10$ of dataset
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['youtube', 'chinese', 'low', 'music', 'home', 'favorite', 'history', 'state', 'download', 'exam', 'video', 'service', 'prepare', 'good', 'movie']


THE TOP 15 WORDS FOR TOPIC #1
['delhi', 'vote', 'presidential', 'america', 'skill', 'problem', 'election', 'hillary', 'clinton', 'improve', 'president', 'donald', 'example', 'win', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['medium', 'blog', 'determine', 'affect', 'message', 'point', 'file', 'business', 'send', 'view', 'online', 'bank', 'black', 'start', 'money']


THE TOP 15 WORDS FOR TOPIC #3
['development', 'overcome', 'apple', 'turn', 'hear', 'amazon', 'know', 'online', 'sell', 'product', 'design', 'pro', 'school', 'mind', 'girl']


THE TOP 15 WORDS FOR TOPIC #4
['degree', 'canada', 'battle', 'program', 'list', 'today', '10', 'choose', 'date', 'age', 'language', 'increase', 'college', 'old', 'year']


THE TOP 15 WORDS FOR TOPIC #5
['c', 'marketing', 'course', 'fall', 'market', 'online', 'language', 'easy', 

In [50]:
#third run with grid search params
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['music', 'web', 'design', 'favorite', 'pro', 'project', 'idea', 'watch', 'cause', 'website', 'increase', 'good', 'company', 'movie', 'people']


THE TOP 15 WORDS FOR TOPIC #1
['speed', 'fact', 'value', 'picture', 'choose', 'light', 'tv', 'interesting', 'power', 'delhi', 'look', 'major', 'university', 'come', 'new']


THE TOP 15 WORDS FOR TOPIC #2
['modi', 'affect', 'help', 'government', 'rupee', 'problem', 'black', 'ban', 'start', '1000', 'business', '500', 'indian', 'note', 'money']


THE TOP 15 WORDS FOR TOPIC #3
['talk', 'short', 'air', 'girlfriend', 'sell', 'password', 'email', 'fall', 'want', 'guy', 'friend', 'person', 'know', 'love', 'girl']


THE TOP 15 WORDS FOR TOPIC #4
['interview', 'follow', 'important', 'mark', 'water', 'pay', 'google', 'people', 'answer', 'old', 'need', 'ask', 'year', 'question', 'quora']


THE TOP 15 WORDS FOR TOPIC #5
['laptop', 'easy', 'service', 'site', 'course', 'learn', 'college', 'exam', 'game', 'prepare', 'online', 'b

In [37]:
%%time
from sklearn.model_selection import GridSearchCV


# Define Search Param
search_params = {'n_components': [15],'learning_decay': [.7], 'batch_size': [128, 2048, 8192]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=25)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, cv=3, verbose=2, n_jobs = -1)

# Do the Grid Search
model.fit(dtm)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed:  5.4min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  6.4min finished


CPU times: user 1min 50s, sys: 693 ms, total: 1min 50s
Wall time: 8min 1s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=25,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                        

In [40]:
from sklearn.model_selection import GridSearchCV


# Define Search Param
search_params = {'n_components': [15],'learning_decay': [.7], 'batch_size': [1024, 2048, 8192]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=25)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, cv=3, verbose=2, n_jobs = -1)

# Do the Grid Search
model.fit(dtm)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:  2.9min remaining:  3.6min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  4.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  4.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=25,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                        

In [41]:
best_lda_model = model.best_estimator_

#model parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

Best Model's Params:  {'batch_size': 8192, 'learning_decay': 0.7, 'n_components': 15}
Best Log Likelihood Score:  -421640.13401414634
Model Perplexity:  1436.7232893681153


In [39]:
best_lda_model = model.best_estimator_

#model parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 15}
Best Log Likelihood Score:  -422416.91315089236
Model Perplexity:  1476.5022152079882


In [53]:
import numpy as np
# Create Document - Topic Matrix
lda_output = lda_model.transform(dtm)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(quoras))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,dominant_topic
Doc0,0.02,0.27,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.27,0.02,0.02,0.02,0.27,1
Doc1,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.17,0.01,0.01,0.7,0.01,0.01,0.01,0.01,10
Doc2,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.36,0.02,0.02,0.02,0.36,10
Doc3,0.08,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.61,0.01,0.01,0.18,11
Doc4,0.02,0.27,0.02,0.02,0.02,0.02,0.27,0.02,0.02,0.02,0.02,0.02,0.27,0.02,0.02,1
Doc5,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.87,0.01,0.01,0.01,0.01,0.01,0.01,8
Doc6,0.03,0.03,0.03,0.53,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,3
Doc7,0.03,0.03,0.03,0.03,0.03,0.53,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,5
Doc8,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.69,0.02,0.02,0.02,0.02,0.02,0.02,0.02,7
Doc9,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.81,0.01,13


In [58]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,5,4391
1,0,4011
2,11,2927
3,8,2907
4,4,2838
5,6,2835
6,7,2815
7,3,2476
8,12,2423
9,2,2247


In [57]:
%%time
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, dtm, cv, mds='tsne')
panel

//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


CPU times: user 12.2 s, sys: 375 ms, total: 12.6 s
Wall time: 1min 2s


PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
10     278.092407 -321.946655       1        1  9.083770
6     -100.485588  -71.434715       2        1  8.353198
13     485.537476  209.835678       3        1  7.486514
3     -456.470703  205.625809       4        1  7.130512
1        2.903878 -343.362335       5        1  7.021491
5     -253.830185  416.522919       6        1  6.921795
14     131.275345  -93.098488       7        1  6.701322
8      259.875946  396.580719       8        1  6.663506
11     428.057343  -85.513237       9        1  6.143288
2     -379.232269  -61.852638      10        1  6.057297
12    -263.977448 -305.579315      11        1  5.839292
7       26.739553  193.840317      12        1  5.738295
0     -197.703110  152.006256      13        1  5.676338
9      242.977280  117.502815      14        1  5.622848
4        5.938293  464.183990      15        1  5.560533, topic_info=     Category         Freq        Term        Total  loglift  logprob
1178  Default  4370.000000        good  4370.000000  30.0000  30.0000
1553  Default  1288.000000        like  1288.000000  29.0000  29.0000
2791  Default  1168.000000         use  1168.000000  28.0000  28.0000
789   Default   851.000000  difference   851.000000  27.0000  27.0000
2131  Default   888.000000       quora   888.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
2261  Topic15   102.171162         rid   148.478728   2.5157  -4.4886
1026  Topic15    95.977573    favorite   140.582115   2.5078  -4.5511
825   Topic15    90.099659         dog   130.448021   2.5194  -4.6143
713   Topic15    87.242579        dark   129.420158   2.4951  -4.6465
561   Topic15    67.146746       color    89.706704   2.5998  -4.9083

[699 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3         6  0.056325        1
3         7  0.398607        1
3         8  0.541586        1
4         7  0.995755       10
6         2  0.997943     1000
...     ...       ...      ...
2932      7  0.329002     year
2932      8  0.001502     year
2938      1  0.197136  youtube
2938      6  0.657121  youtube
2938     10  0.137995  youtube

[940 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 7, 14, 4, 2, 6, 15, 9, 12, 3, 13, 8, 1, 10, 5])

#### TASK: Add a new column to the original quora dataframe that labels each question into one of the topic categories.

In [59]:
topic_results = lda_model.transform(dtm)

In [60]:
dtm.shape

(40429, 2948)

In [62]:
topic_results.shape

(40429, 15)

In [63]:
topic_results[0].round(2)

array([0.02, 0.27, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.27,
       0.02, 0.02, 0.02, 0.27])

In [64]:
topic_results[0].argmax()

14

In [65]:
topic_results.argmax(axis=1)

array([14, 10, 14, ...,  4, 14,  6])

In [66]:
quoras['Topic'] = topic_results.argmax(axis=1)

In [68]:
quoras.head(10)

,Question,Topic
208679,Can Kodi work on my visio smart tv?,14
227891,Who suffers more due to tension between wife a...,10
93705,Is cloud mining even profitable?,14
45579,Why do people go back to school for a second b...,11
250899,What are some places in Lahore to do a food dr...,1
235918,Why is the number of my app downloads rapidly ...,8
104616,Why does penis bent?,3
104441,How do you format a Nokia 5800?,5
128715,What are the most dangerous neighborhoods in C...,7
29131,If you could change anything about today's sys...,13


# Great job!

In [ ]:
#meeting on results